In [2]:
import numpy as np 

def generate_key(w,m,n):
    S = (np.random.rand(m,n)*w/(2**16)) 
    return S


def encrypt(x,S,m,n,w):
    assert len(x) == len(S)
    e = (np.random.rand(m))
    c = np.linalg.inv(S).dot((w*x) +e)  #np.linalg.inv 矩阵求逆 
    return c


def decrypt(c,S,w):
    return (S.dot(c)/w).astype('int')


def get_c_star(c,m,l):
    c_star = np.zeros(l*m,dtype='int')
    for i in range(m):
        b = np.array(list(np.binary_repr(np.abs(c[i]))),dtype='int') #np.binary_repr 函数返回给定宽度中十进制数的二进制表示
        if(c[i] <0):
            b *= -1
            c_star[(i*l) + (l -len(b)):(i+1)*l] += b 
    return c_star

def get_S_star(S,m,n,l):
    S_star = list()
    for i in range(l):
        S_star.append(S*2**(l-i-1))
    S_star = np.array(S_star).transpose(1,2,0).reshape(m,n*l) # 转置 ,高维张量转置
    return S_star

 
x = np.array([0,1,2,5])

 
m = len(x)
n = m 
w = 16

S = generate_key(w,m,n)
 
print(' S :',S)

 S : [[3.33000594e-05 1.96967841e-04 1.58901145e-05 9.10829351e-05]
 [4.53317739e-05 1.94112511e-04 1.73038286e-04 9.95281018e-05]
 [1.93546138e-04 3.90193828e-05 2.13092535e-04 8.81439258e-05]
 [1.22937252e-04 1.84516649e-04 2.80538596e-06 1.56489098e-04]]


In [3]:
c = encrypt(x,S,m,n,w)
print(' c :',c)

 c : [-545759.12929426 -748584.7452061    29114.5687089  1823047.44579441]


In [4]:
print(' decrypt c :', decrypt(c,S,w))

 decrypt c : [0 1 2 5]


In [5]:
print(' decrypt (c + c) :' ,decrypt(c+c,S,w))

 decrypt (c + c) : [ 0  2  4 10]


In [6]:
print('decrypt (c*10) :',decrypt(c*10,S,w))

decrypt (c*10) : [ 0 10 20 50]


In [7]:
print("#################### Optimizing Encryption ########################")

#################### Optimizing Encryption ########################


In [13]:
import numpy as np 

def generate_key(w,m,n):
    S = (np.random.rand(m,n)*w/(2**16))
    return S


def encrypt(x,S,m,n,w):
    assert len(x) == len(S)
    
    e = (np.random.rand(m))
    c = np.linalg.inv(S).dot((w*x) +e)
    return c


def decrypt(c,S,w):
    return (S.dot(c)/w).astype('int')

 
def get_c_star(c,m,l): # 生成辅助变量，有助于快速计算
    c_star = np.zeros(l*m,dtype='int')
    for i in range(m):
        b = np.array(list(np.binary_.repr(np.abs(c[i]))),dtype='int')
        if(c[i]<0):
            b *= -1
        c_star[(i*l) + (l - len(b)):(i+1)*l] += b
    
    return c_star


def switch_key(c,S,m,n,T): 
    l = int(np.ceil(np.log2(np.max(np.abs(c)))))
    c_star = get_c_star(c,m,l)
    S_star = get_S_star(S,m,n,l)
    n_prime = n +1
    
    S_prime = np.concatenate((np.eye(m),T.T),0).T #concatenate 连接元素
    A = (np.random.rand(n_prime -m,n*l)*10).astype('int')
    E = (1*np.random.rand(S_star.shape[0],S_star.shape[1])).astype('int')
    M = np.concatenate(((S_star - T.dot(A) + E),A),0)
    c_prime = M.dot(c_star)
    return c_prime,S_prime

def get_S_star(S,m,n,l): # 生成辅助变量，有助于快速计算
    S_star = list()
    for i in range(l):
        S_star.append(S*2**(l-i-1))
    S_star = np.array(S_star).transpose(1,2,0).reshape(m,n*l)
    return S_star

def get_T(n):
    n_prime = n +1
    T = (10*np.random.rand(n,n_prime-n)).astype('int')
    return T

def encrypt_via_switch(x,w,m,n,T):
    c,S = switch_key(x*w,np.eye(m),m,n,T)
    return c,S

 
 
x = np.array([0,1,2,5])
m = len(x)
n = m
w = 16
S = generate_key(w,m,n) 
print('S :',S)

S : [[9.48725856e-05 1.76495317e-04 2.03972659e-04 1.91701357e-04]
 [1.83401523e-04 5.02720860e-05 1.35012696e-05 6.97875470e-05]
 [2.40691134e-04 2.39303461e-05 6.07491583e-05 7.57130726e-05]
 [8.30486449e-05 6.11973887e-05 6.02494789e-05 1.32882347e-04]]


In [9]:
T = get_T(n)
print(' T :',T)

 T : [[7]
 [8]
 [7]
 [3]]


In [10]:
c,S = encrypt_via_switch(x,w,m,n,T)
print(' C :',c)
print(' S :',S)

 C : [-35. -24.  -3.  65.   5.]
 S : [[1. 0. 0. 0. 7.]
 [0. 1. 0. 0. 8.]
 [0. 0. 1. 0. 7.]
 [0. 0. 0. 1. 3.]]


In [11]:
print(' decrypt(c + c ,S,w) :',decrypt(c + c ,S,w) )

 decrypt(c + c ,S,w) : [ 0  2  4 10]


In [12]:
print('decrypt(c*10,S,w) :', decrypt(c*10,S,w))

decrypt(c*10,S,w) : [ 0 10 20 50]


In [ ]:
T = get_T(n)
c,S = encrypt_via_switch(x,w,m,n,T)
decrypt(c+c,S,w)
decrypt(c*10,S,w)
